In [1]:
import pylab
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pprint as pp
import numpy as np
import document_processor as dp
import test
import classifier
from LOD_extraction import entities_distribution
from text_utils import TextUtils
from mongo_hc import MongoHC
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import HTML
from IPython.display import Math 
from alchemyapi_python.alchemyapi import AlchemyAPI
from scipy.cluster import hierarchy as hr


In [ ]:
def entities_distribution(db, dataset):
    mongo = MongoHC(db, dataset)

    data = [doc for doc in mongo.get_all(order_by='id_doc')]

    entities = set()

    for d in data[:]:
        for e in d['dandelion']['annotations']:
            entities.add(e['lod']['dbpedia'])
    entities_dict = {e: 0 for i, e in enumerate(entities)}

    for d in data:
        for e in d['dandelion']['annotations']:
            entities_dict[e['lod']['dbpedia']] += 1

    return entities_dict, entities

In [ ]:
mongo = MongoHC('hc', 're0')

data = [doc for doc in mongo.get_all(order_by='id_doc')]

entities = []

for d in data[:]:
    for e in d['dandelion']['annotations']:
        entities.append(e['confidence'])

In [ ]:
my_series = pd.Series(entities)

In [ ]:
sns.distplot(my_series);
sns.plt.show()

In [ ]:
only_text = [doc['text'] for doc in data]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5,
                                   max_features=200000,
                                   min_df=2,
                                   stop_words='english',
                                   strip_accents='unicode',
                                   use_idf=True,
                                   ngram_range=(1, 1),
                                   norm='l2',
                                   tokenizer=TextUtils.tokenize_and_stem)

tfidf_matrix = tfidf_vectorizer.fit_transform(only_text)

In [ ]:
df_2 = pd.DataFrame(tfidf_matrix.data)

In [ ]:
sns.distplot(df_2);
sns.plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True, figsize=(12,4))
#ax1
sns.distplot(my_series, ax=ax1)
#ax2
sns.distplot(df_2, ax=ax2);

#ax3
sns.distplot(my_series, ax=ax3)
sns.distplot(df_2, ax=ax3);
ax3.set_xlim([-0.2,1])
sns.plt.show()

In [ ]:
fig, (ax1) = plt.subplots(1,1, sharey=True, figsize=(15.0, 8.0))
a = sns.distplot(my_series, ax=ax1, label='distribuzione delle entita\'')
b = sns.distplot(df_2, ax=ax1, label='distribuzione dei termini');
ax3.set_xlim([-0.1,1])
legend = plt.legend()
sns.plt.show()

In [2]:
mongo_test = MongoHC('hc', 'test_new')

In [3]:
data = [res for res in mongo_test.get_by_key('dataset', 're0', order_by='n_attempt')]
test_re0 = pd.DataFrame(data)
test_re0[test_re0['avg_f_score'] == test_re0['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
22,0.92,0.656501


In [ ]:
re0 = sns.barplot(x='gamma', y='avg_f_score', data=test_re0,
                palette=sns.color_palette("Reds", n_colors=test_re0.shape[0]))
sns.plt.show()

In [4]:
data = [res for res in mongo_test.get_by_key('dataset', 're1', order_by='n_attempt')]
test_re1 = pd.DataFrame(data)
test_re1[test_re1['avg_f_score'] == test_re1['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
23,0.93,0.785931


In [5]:
data = [res for res in mongo_test.get_by_key('dataset', 're0_lsa', order_by='n_attempt')]
test_re0_lsa = pd.DataFrame(data)
test_re0_lsa[test_re0_lsa['avg_f_score'] == test_re0_lsa['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
15,0.85,0.665929


In [6]:
data = [res for res in mongo_test.get_by_key('dataset', 're1_lsa', order_by='n_attempt')]
test_re1_lsa = pd.DataFrame(data)
test_re1_lsa[test_re1_lsa['avg_f_score'] == test_re1_lsa['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
2,0.72,0.779976


In [7]:
data = [res for res in mongo_test.get_by_key('dataset', 're1_abstract_lsa', order_by='n_attempt')]
test_re1_abstract_lsa = pd.DataFrame(data)
test_re1_abstract_lsa[test_re1_abstract_lsa['avg_f_score'] == test_re1_abstract_lsa['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
2,0.72,0.802188


In [70]:
data = [res for res in mongo_test.get_by_key('dataset', 're0_abstract_lsa', order_by='n_attempt')]
test_re0_abstract_lsa = pd.DataFrame(data)
test_re0_abstract_lsa.shape
test_re0_abstract_lsa[test_re0_abstract_lsa['avg_f_score'] == test_re0_abstract_lsa['avg_f_score'].max()][['gamma', 'avg_f_score']]

,gamma,avg_f_score
19,0.89,0.663865


In [ ]:
re1 = sns.barplot(x='gamma', y='avg_f_score', data= test_re1,
                palette=sns.color_palette("Reds", n_colors=test_re1.shape[0]))
sns.plt.show()

In [9]:
reds = sns.color_palette("Reds_d")
greens = sns.color_palette("Greens_d")
blues = sns.color_palette("Blues_d")

In [ ]:
sns.plt.plot([0.5] * 100)

sns.plt.show()

In [69]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, sharey=True, figsize=(15.0, 10.0))

clrs = [reds[4] if (x < test_re0['avg_f_score'].max()) else reds[0] for x in test_re0['avg_f_score'] ]
a = sns.barplot(x='gamma', y='avg_f_score', data=test_re0, ax=ax1, label='prova1', palette=clrs, ci=0.2)

clrs = [reds[4] if (x < test_re1['avg_f_score'].max()) else reds[0] for x in test_re1['avg_f_score'] ]
b = sns.barplot(x='gamma', y='avg_f_score', data=test_re1, ax=ax2, label='prova', palette=clrs)

clrs = [blues[4] if (x < test_re0_lsa['avg_f_score'].max()) else blues[0] for x in test_re0_lsa['avg_f_score'] ]
c = sns.barplot(x='gamma', y='avg_f_score', data=test_re0_lsa, ax=ax3, label='prova',palette=clrs)
clrs = [blues[4] if (x < test_re1_lsa['avg_f_score'].max()) else blues[0] for x in test_re1_lsa['avg_f_score'] ]
d = sns.barplot(x='gamma', y='avg_f_score', data=test_re1_lsa, ax=ax4, label='prova',palette=clrs)

clrs = [greens[4] if (x < test_re0_abstract_lsa['avg_f_score'].max()) else greens[0] for x in test_re0_abstract_lsa['avg_f_score'] ]     
e = sns.barplot(x='gamma', y='avg_f_score', data=test_re0_abstract_lsa, ax=ax5, label='prova',palette=clrs)
clrs = [greens[4] if (x < test_re1_abstract_lsa['avg_f_score'].max()) else greens[0] for x in test_re1_abstract_lsa['avg_f_score'] ]     
f = sns.barplot(x='gamma', y='avg_f_score', data=test_re1_abstract_lsa, ax=ax6, label='prova',palette=clrs)

plt.text(0.1, 0.9,'re0 testo + entita\'', ha='left', va='center', transform=a.transAxes)
plt.text(0.1, 0.9,'re1 testo + entita\'', ha='left', va='center', transform=b.transAxes)
plt.text(0.1, 0.9,'re0 testo + entita\' con LSA', ha='left', va='center', transform=c.transAxes)
plt.text(0.1, 0.9,'re1 testo + entita\' con LSA', ha='left', va='center', transform=d.transAxes)
plt.text(0.1, 0.9,'re0 testo + entita\' + abstract con LSA', ha='left', va='center', transform=e.transAxes)
plt.text(0.1, 0.9,'re1 testo + entita\' + abstract con LSA', ha='left', va='center', transform=f.transAxes)

for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    ax.set_ylim([0,1])

for g in [a, b, c, d, e, f]:
    g.set_xticklabels(test_re0['gamma'], rotation=90)
    g.set(xlabel='gamma', ylabel='FScore')
    
#legend = plt.legend()
fig.tight_layout()

sns.plt.show()

In [2]:
linkage_matrix_re0 = classifier.cluster_dandelion_2('re0')

tfifd matrix dimension: 1262 x 4446
entities matrix dimension: 1262 x 2911 
non zero elements in entities matrix: 6538
[0.21428571428571427,
 0.6618705035971224,
 0.6619718309859155,
 0.846153846153846,
 0.6606606606606606,
 0.6268656716417911,
 0.8735632183908045,
 0.888888888888889,
 0.592964824120603,
 0.32432432432432434,
 0.8,
 0.7353463587921847,
 0.6486486486486486]


In [63]:
sns.clustermap(linkage_matrix_re0)
sns.plt.show()

In [71]:
linkage_matrix_re1 = linkage_matrix = classifier.cluster_dandelion_2('re1')

tfifd matrix dimension: 1404 x 5660
entities matrix dimension: 1404 x 4498 
non zero elements in entities matrix: 7344
[0.6857142857142858,
 0.5882352941176471,
 0.8333333333333334,
 0.9308755760368664,
 0.7610619469026549,
 0.7906976744186047,
 0.8623595505617978,
 1.0,
 0.5625000000000001,
 0.8165680473372782,
 0.37333333333333335,
 0.6760563380281689,
 0.5714285714285715,
 0.22222222222222224,
 0.8571428571428571,
 0.6666666666666667,
 0.9473684210526316,
 0.6486486486486486,
 0.7213114754098361,
 0.8058823529411765,
 0.8923076923076924,
 0.7599999999999999,
 0.7352941176470588,
 0.3448275862068965]


In [14]:
dendogram = hr.dendrogram(linkage_matrix_re0, p=10, truncate_mode='level', labels=[' '] * linkage_matrix_re0.shape[0]*10)
plt.show()